In [1]:
# Install libraries within the notebook scope
sc.install_pypi_package("boto3")
sc.install_pypi_package("pandas")
sc.install_pypi_package("requests")
sc.install_pypi_package("s3fs")
sc.install_pypi_package("fsspec")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1603597063561_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…




  Running setup.py bdist_wheel for wrapt: started
  Running setup.py bdist_wheel for wrapt: finished with status 'done'
  Stored in directory: /var/lib/livy/.cache/pip/wheels/b1/c2/ed/d62208260edbd3fa7156545c00ef966f45f2063d0a84f8208a
  Running setup.py bdist_wheel for aiohttp: started
  Running setup.py bdist_wheel for aiohttp: finished with status 'done'
  Stored in directory: /var/lib/livy/.cache/pip/wheels/91/27/92/c0c7d88b626c5497188729c9c0e5d17d21c2c38c5d7acca78b
  Running setup.py bdist_wheel for multidict: started
  Running setup.py bdist_wheel for multidict: finished with status 'error'
  Complete output from command /tmp/1603597215298-0/bin/python -u -c "import setuptools, tokenize;__file__='/mnt/tmp/pip-build-730hag_t/multidict/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/tmpybbztl_ypip-wheel- --python-tag cp37:
  **********************
  * Accellerated build 

In [28]:
import boto3
from datetime import datetime
import fsspec
import pandas as pd
from pyspark.sql.dataframe import DataFrame
from pyspark.sql import functions as f, types as t, Window
from pathlib import Path
import re
import requests
import s3fs
import subprocess
import timeit
from urllib.parse import urlparse

# Removes truncation of columns, column values in Pandas
# by default
pd.set_option('max_columns', None)
pd.set_option('max_colwidth', None)

# Monkey patching the DataFrame transform method for Spark 2.4
# This is available by default in Spark 3.0
def transform(self, f):
    return f(self)
DataFrame.transform = transform

# Override the timeit template to return the command's
# return value in addition to the time
# Reference: https://stackoverflow.com/questions/24812253/how-can-i-capture-return-value-with-python-timeit-module
timeit.template = """
def inner(_it, _timer{init}):
    {setup}
    _t0 = _timer()
    for _i in _it:
        retval = {stmt}
    _t1 = _timer()
    return _t1 - _t0, retval
"""

def shell_cmd(cmd):
    """
    Wrapper for running shell commands and printing the output
    Some helpful recipes:
    - List files on hdfs: shell_cmd("hdfs dfs -ls hdfs:///tmp/data/")
    - Remove files from hdfs: shell_cmd("hdfs dfs -rm -r hdfs:///tmp/data/test_pyspark")
    """
    for line in subprocess.check_output(cmd, shell=True).split(b'\n'):
        print(line)

def timer_method(cmd):
    """
    Wrapper for timeit that returns the value of a function and its runtime
    To use, pass a string of the function you wish to time
    Example: 
     run_time, result = timer_method("myfunction(arg1, arg2)")
    """
    # Setting globals = globals() enables the timeit function
    # to return the value generated by cmd
    return timeit.timeit(cmd, number=1, globals = globals())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Set your s3 bucket name
This should be data-scale-oreilly-{your name}   
If you dont remember check the [S3 console](https://s3.console.aws.amazon.com/)

In [3]:
MY_BUCKET_NAME = "data-scale-oreilly"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Ingesting from an S3 bucket - NYC Taxi Data

https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page
* Taxi data 
* Data dictionaries 
* Taxi zone lookup table

Take a few minutes to look through the Data Dictionaries and Metadata and the Taxi Zone Maps and Lookup Tables. What are some things you notice about the data?

Data ingestion has the ultimate goal of collecting, aggregating, and surfacing data for a specific purpose; an analysis, an API, a dashboard, etc. Think about how you might use the taxi data to answer the following questions:

1. Which borough is the most popular pickup or drop off spot?
1. Are green taxis more popular for trips within the same borough vs yellow taxis?
1. Build a recommendation engine that predicts surge pricing for a given time of day based on historical data  

With this in mind, lets work through bringing this data onto the cluster

In [4]:
# Note, if you copy the link from the taxi data website you will see:
# https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-01.csv
# Two things - first, the portion of the URL following "aws.com" is the 
# bucket name. Second, in "trip+data" the "+" is a space
taxi_data_path = "s3://nyc-tlc/trip data/yellow_tripdata_2020-01.csv"

# When working with big data it can be challenging to view the data. How would you 
# go about getting a sample of this data? (download it, use requests, pandas, etc)

# Pandas uses s3fs to read_csv from s3:
pd.read_csv(taxi_data_path, keep_default_na=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Take a look at the data. Notice how pandas will try to assign types. Is this desirable?
# Why or why not?
# Since we have column names it also seems this data has a header
pd_df_taxi.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VendorID                  object
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count           object
trip_distance            float64
RatecodeID                object
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type              object
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
dtype: object

In [11]:
# For reference, look at the Spark DataFrameReader, csv:
# https://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html
ps_df_taxi = spark.read.option('header', True).csv(taxi_data_path)

# Talk through the spark UI here, partcularly note that the cell number will show up in the SparkUI
# Job list next to the Job id. For example. this would be Job 2 (cell number)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
ps_df_taxi.show(10, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|1       |2020-01-01 00:28:15 |2020-01-01 00:33:03  |1              |1.20         |1         |N                 |238         |239         |1           |6          |3    |0.5    |1.47      |0           |0.3                  

In [7]:
# Lets see how the spark dataframe reader interpreted the data
# Talk about nullable vs non nullable and maybe a small bit about data schemas
ps_df_taxi.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- congestion_surcharge: string (nullable = true)

In [8]:
# Talk through ingest practices around retaining original data vs augmenting
# For example, we may want to keep the data in its default format so we can
# refer back to it if there are bugs in our data ingestion code
ps_df_taxi.write.option("header", True).csv("hdfs:///tmp/input/taxi_data")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'path hdfs://ip-172-31-6-49.us-east-2.compute.internal:8020/tmp/input/taxi_data already exists.;'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/readwriter.py", line 932, in csv
    self._jwrite.csv(path)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'path hdfs://ip-172-31-6-49.us-east-2.compute.internal:8020/tmp/input/taxi_data already exists.;'



In [9]:
# Discuss how spark writes files out
shell_cmd("hdfs dfs -ls hdfs:///tmp/input/taxi_data")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

b'Found 9 items'
b'-rw-r--r--   1 livy hadoop          0 2020-10-18 23:28 hdfs:///tmp/input/taxi_data/_SUCCESS'
b'-rw-r--r--   1 livy hadoop   73917327 2020-10-18 23:28 hdfs:///tmp/input/taxi_data/part-00000-4ec8700e-82dd-454f-a072-d3a2943c8375-c000.csv'
b'-rw-r--r--   1 livy hadoop   73919562 2020-10-18 23:28 hdfs:///tmp/input/taxi_data/part-00001-4ec8700e-82dd-454f-a072-d3a2943c8375-c000.csv'
b'-rw-r--r--   1 livy hadoop   73920337 2020-10-18 23:28 hdfs:///tmp/input/taxi_data/part-00002-4ec8700e-82dd-454f-a072-d3a2943c8375-c000.csv'
b'-rw-r--r--   1 livy hadoop   73921216 2020-10-18 23:28 hdfs:///tmp/input/taxi_data/part-00003-4ec8700e-82dd-454f-a072-d3a2943c8375-c000.csv'
b'-rw-r--r--   1 livy hadoop   73918682 2020-10-18 23:28 hdfs:///tmp/input/taxi_data/part-00004-4ec8700e-82dd-454f-a072-d3a2943c8375-c000.csv'
b'-rw-r--r--   1 livy hadoop   73921304 2020-10-18 23:28 hdfs:///tmp/input/taxi_data/part-00005-4ec8700e-82dd-454f-a072-d3a2943c8375-c000.csv'
b'-rw-r--r--   1 livy hadoop  

In [15]:
# Examine the data, what do you notice?
column_subset = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'PULocationID', 'DOLocationID', 'fare_amount', 'tip_amount']
ps_df_taxi.select(*column_subset).show(10, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------------+---------------+-------------+------------+------------+-----------+----------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|PULocationID|DOLocationID|fare_amount|tip_amount|
+--------------------+---------------------+---------------+-------------+------------+------------+-----------+----------+
|2020-01-01 00:28:15 |2020-01-01 00:33:03  |1              |1.20         |238         |239         |6          |1.47      |
|2020-01-01 00:35:39 |2020-01-01 00:43:04  |1              |1.20         |239         |238         |7          |1.5       |
|2020-01-01 00:47:41 |2020-01-01 00:53:52  |1              |.60          |238         |238         |6          |1         |
|2020-01-01 00:55:23 |2020-01-01 01:00:14  |1              |.80          |238         |151         |5.5        |1.36      |
|2020-01-01 00:01:58 |2020-01-01 00:04:16  |1              |.00          |193         |193         |3.5        |0         |
|2020-01

In [11]:
# Keep in mind the datatype when considering these results
ps_df_taxi.select(*column_subset).describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+---------------------+------------------+------------------+------------------+-----------------+------------------+------------------+
|summary|tpep_pickup_datetime|tpep_dropoff_datetime|   passenger_count|     trip_distance|      PULocationID|     DOLocationID|       fare_amount|        tip_amount|
+-------+--------------------+---------------------+------------------+------------------+------------------+-----------------+------------------+------------------+
|  count|             6405008|              6405008|           6339567|           6405008|           6405008|          6405008|           6405008|           6405008|
|   mean|                null|                 null|1.5153326717739555|2.9296439333096704|164.73225778952968|162.6626908194338|  12.6941081197704|2.1893418306424284|
| stddev|                null|                 null|1.1515942134278123|  83.1591059732504| 65.54373944111667| 69.9126062949612|12.127295340046542| 2.760028392378421|
|   

In [12]:
# Casting columns to a type
(ps_df_taxi.select("passenger_count")
 .withColumn("passenger_count", f.col("passenger_count").cast(t.IntegerType()))
).distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+
|passenger_count|
+---------------+
|           null|
|              1|
|              6|
|              3|
|              4|
|              8|
|              5|
|              2|
|              7|
|              0|
|              9|
+---------------+

In [13]:
# Casting pandas columns to a type - this will give an error on empty cells
(pd_df_taxi[[*column_subset]]
         .astype({'passenger_count': 'Int64'}))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

invalid literal for int() with base 10: ''
Traceback (most recent call last):
  File "/tmp/1603064188681-0/lib/python3.7/site-packages/pandas/core/generic.py", line 5531, in astype
    col.astype(dtype=dtype[col_name], copy=copy, errors=errors)
  File "/tmp/1603064188681-0/lib/python3.7/site-packages/pandas/core/generic.py", line 5546, in astype
    new_data = self._mgr.astype(dtype=dtype, copy=copy, errors=errors,)
  File "/tmp/1603064188681-0/lib/python3.7/site-packages/pandas/core/internals/managers.py", line 595, in astype
    return self.apply("astype", dtype=dtype, copy=copy, errors=errors)
  File "/tmp/1603064188681-0/lib/python3.7/site-packages/pandas/core/internals/managers.py", line 406, in apply
    applied = getattr(b, f)(**kwargs)
  File "/tmp/1603064188681-0/lib/python3.7/site-packages/pandas/core/internals/blocks.py", line 595, in astype
    values = astype_nansafe(vals1d, dtype, copy=True)
  File "/tmp/1603064188681-0/lib/python3.7/site-packages/pandas/core/dtypes/cast.

In [14]:
# To convert to Integer using pandas, we have to first deal with the null values
# to_numeric with 'coerce' will fill invalid integer values with np.NaN
# the Int64 type in later versions of pandas will convert np.NaN to a nullable
# integer type: https://pandas.pydata.org/pandas-docs/stable/user_guide/integer_na.html
pd.to_numeric(pd_df_taxi.passenger_count, errors='coerce').astype('Int64').unique()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<IntegerArray>
[1, 4, 2, 3, 6, 5, 0, 8, 7, 9, <NA>]
Length: 11, dtype: Int64

### Lab 2.1 - Transform the column_subset of taxi data to data types that accurately represent the data

Result: a transformed_taxi dataframe with the column_subset columns cast to an appropriate type

Available types are listed in the pyspark.sql.types module https://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html#module-pyspark.sql.types  

This is imported as `t`, so to apply the IntegerType use `t.IntegerType()`

For pandas, see the following resources on converting types
https://stackoverflow.com/questions/15891038/change-column-type-in-pandas

In [16]:
# Talk about transforming
# the data into a type schema to surface for analytical operations - i.e. pay the penalty of time
# on the ingest to transform strings to timestamps so the analysis side can use datetime methods
# without having to remember to cast

# Any gotchyas on transforming types we should discuss? Perhaps datetime casting
def transform_taxi_ps(ps_df_taxi):
    res = (ps_df_taxi
            .select(*column_subset)
            .withColumn("tpep_pickup_datetime", f.col("tpep_pickup_datetime").cast(t.TimestampType()))
            .withColumn("tpep_dropoff_datetime", f.col("tpep_dropoff_datetime").cast(t.TimestampType()))
            .withColumn("passenger_count", f.col("passenger_count").cast(t.IntegerType()))
            .withColumn("trip_distance", f.col("trip_distance").cast(t.FloatType()))
            .withColumn("PULocationID", f.col("PULocationID").cast(t.IntegerType()))
            .withColumn("DOLocationID", f.col("DOLocationID").cast(t.IntegerType()))
            .withColumn("fare_amount", f.col("fare_amount").cast(t.FloatType()))
            .withColumn("tip_amount", f.col("tip_amount").cast(t.FloatType()))             
           )
    res.count()  # Run this to force pyspark to collect the data
    return res

def transform_taxi_pd(pd_df_taxi):
    res = pd_df_taxi[[*column_subset]].copy()
    res.tpep_pickup_datetime = pd.to_datetime(res.tpep_pickup_datetime)
    res.tpep_dropoff_datetime = pd.to_datetime(res.tpep_dropoff_datetime)
    res.passenger_count = pd.to_numeric(res.passenger_count, errors='coerce').astype('Int64')
    res.trip_distance = pd.to_numeric(res.trip_distance, errors='coerce')
    res.PULocationID = pd.to_numeric(res.PULocationID, errors='coerce').astype('Int64')
    res.DOLocationID = pd.to_numeric(res.DOLocationID, errors='coerce').astype('Int64')
    res.fare_amount = pd.to_numeric(res.fare_amount, errors='coerce')
    res.tip_amount = pd.to_numeric(res.tip_amount, errors='coerce')
    res.count()
    return res



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
ps_run_time, ps_transform_taxi = timer_method("transform_taxi_ps(ps_df_taxi)")
pd_run_time, pd_transform_taxi = timer_method("transform_taxi_pd(pd_df_taxi)")
print(f"pyspark runtime: {ps_run_time} pandas runtime {pd_run_time}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
pd_transform_taxi.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'pd_transform_taxi' is not defined
Traceback (most recent call last):
NameError: name 'pd_transform_taxi' is not defined



In [19]:
ps_transform_taxi.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------------+---------------+-------------+------------+------------+-----------+----------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|PULocationID|DOLocationID|fare_amount|tip_amount|
+--------------------+---------------------+---------------+-------------+------------+------------+-----------+----------+
| 2020-01-01 00:28:15|  2020-01-01 00:33:03|              1|          1.2|         238|         239|        6.0|      1.47|
| 2020-01-01 00:35:39|  2020-01-01 00:43:04|              1|          1.2|         239|         238|        7.0|       1.5|
| 2020-01-01 00:47:41|  2020-01-01 00:53:52|              1|          0.6|         238|         238|        6.0|       1.0|
| 2020-01-01 00:55:23|  2020-01-01 01:00:14|              1|          0.8|         238|         151|        5.5|      1.36|
| 2020-01-01 00:01:58|  2020-01-01 00:04:16|              1|          0.0|         193|         193|        3.5|       0.0|
+-------

In [19]:
# Note that this only shows results for numeric and string columns - now that the data has been cast to 
# types, we can explore it a bit more
ps_transform_taxi.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+------------------+------------------+-----------------+------------------+------------------+
|summary|   passenger_count|     trip_distance|      PULocationID|     DOLocationID|       fare_amount|        tip_amount|
+-------+------------------+------------------+------------------+-----------------+------------------+------------------+
|  count|           6339567|           6405008|           6405008|          6405008|           6405008|           6405008|
|   mean|1.5153326717739555|2.9296439317743554|164.73225778952968|162.6626908194338|12.694108121822374|2.1893418282101753|
| stddev| 1.151594213427813| 83.15910301291039| 65.54373944111757|69.91260629496095|12.127295342892479|2.7600283861848287|
|    min|                 0|            -30.62|                 1|                1|           -1238.0|             -91.0|
|    max|                 9|         210240.06|               265|              265|            4265.0|            1100.0|
+-------+-------

In [20]:
# Take a look at the date ranges...
ps_transform_taxi.select("tpep_pickup_datetime").sort(f.asc("tpep_pickup_datetime")).show(10, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|tpep_pickup_datetime|
+--------------------+
|2003-01-01 00:07:17 |
|2008-12-31 23:02:40 |
|2008-12-31 23:02:50 |
|2008-12-31 23:03:44 |
|2008-12-31 23:03:48 |
|2008-12-31 23:06:13 |
|2008-12-31 23:17:15 |
|2008-12-31 23:24:11 |
|2008-12-31 23:34:13 |
|2008-12-31 23:35:00 |
+--------------------+
only showing top 10 rows

In [21]:
# Write out transformed data to EBS
ps_transform_taxi.write.mode("append").json("hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Lab 2.2 - Write an ingestion function that does the following:
Given a file path to a taxi data csv (i.e. s3://nyc-tlc/trip data/green_tripdata_2020-01.csv)
1. Read the file into a Spark dataframe
2. Transform the column_subset
3. Write the data as json to hdfs in append mode to `hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json`

Function signature:  
`def ingest_taxi_data(file_name)`

Inputs can be created from:  
`taxi_data_prefix = "s3://nyc-tlc/trip data/"
taxi_data_files = ["yellow_tripdata_2019-01.csv", "yellow_tripdata_2018-01.csv", "yellow_tripdata_2017-01.csv"]  `

In [5]:
def ingest_taxi_data(file_name):
    (spark
         .read
         .option('header', True).csv(file_name)
         .select(*column_subset)
         .withColumn("tpep_pickup_datetime", f.col("tpep_pickup_datetime").cast(t.TimestampType()))
         .withColumn("tpep_dropoff_datetime", f.col("tpep_dropoff_datetime").cast(t.TimestampType()))
         .withColumn("passenger_count", f.col("passenger_count").cast(t.IntegerType()))
         .withColumn("trip_distance", f.col("trip_distance").cast(t.FloatType()))
         .withColumn("PULocationID", f.col("PULocationID").cast(t.IntegerType()))
         .withColumn("DOLocationID", f.col("DOLocationID").cast(t.IntegerType()))
         .withColumn("fare_amount", f.col("fare_amount").cast(t.FloatType()))
         .withColumn("tip_amount", f.col("tip_amount").cast(t.FloatType()))             
         .write
         .mode("append")
         .json("hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json")
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Run the ingest for several files
taxi_data_prefix = "s3://nyc-tlc/trip data/"
taxi_data_files = ["yellow_tripdata_2019-01.csv", "yellow_tripdata_2018-01.csv", "yellow_tripdata_2017-01.csv"]
for file_name in taxi_data_files: 
    taxi_data_path = f"{taxi_data_prefix}{file_name}"
    ingest_taxi_data(taxi_data_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df = spark.read.json("hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json")
df.printSchema()

# Talk through how the read.json interpreted the Integers as Longs, set stage for using schemas

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- DOLocationID: long (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- ingested_on: string (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- trip_distance: double (nullable = true)

### Testing ingestion code

The `ingest_taxi_data` method is not well structured for testing:
* Writes to the file system
* Requires an input file to test
* What other shortcomings?

To make this code more testable, split out the transformation logic so it can be unit tested.  
Definining a transformation function that takes a dataframe and returns a dataframe provides a better interface for unit testing, and a more extensible structure in case we need to add more dataframe functions before or after the transformation step.

In [8]:
def transform_taxi_data(df):
    return (df.withColumn("tpep_pickup_datetime", f.col("tpep_pickup_datetime").cast(t.TimestampType()))
     .withColumn("tpep_dropoff_datetime", f.col("tpep_dropoff_datetime").cast(t.TimestampType()))
     .withColumn("passenger_count", f.col("passenger_count").cast(t.IntegerType()))
     .withColumn("trip_distance", f.col("trip_distance").cast(t.FloatType()))
     .withColumn("PULocationID", f.col("PULocationID").cast(t.IntegerType()))
     .withColumn("DOLocationID", f.col("DOLocationID").cast(t.IntegerType()))
     .withColumn("fare_amount", f.col("fare_amount").cast(t.FloatType()))
     .withColumn("tip_amount", f.col("tip_amount").cast(t.FloatType())))
     
def ingest_taxi_data_transform(file_name):
    # Requires patching of Dataframe.transform method in Spark 2.4, but available natively
    # in Spark 3.0 https://mungingdata.com/pyspark/chaining-dataframe-transformations/
    df_input = (spark
         .read
         .option('header', True).csv(taxi_data_path)
         .select(*column_subset)
         .transform(transform_taxi_data)
         .write
         .mode("append")
         .json("hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json")
    )
    
def ingest_taxi_data_method(file_name):
    # Equivalent code without using the monkey-patched transform method for DataFrame
    df_input = (spark
         .read
         .option('header', True).csv(taxi_data_path)
         .select(*column_subset))
    
    (transform_taxi_data(df_input)
         .write
         .mode("append")
         .json("hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json")
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
test_data = [
    "{'tpep_pickup_datetime': '2020-05-23 21:05:23', 'tpep_dropoff_datetime': '2020-05-23 08:05:23', 'passenger_count': 0, 'trip_distance': 10.5, 'PULocationID': 1, 'DOLocationID': 254, 'fare_amount': 0.05, 'tip_amount': 1.00}",
    "{'tpep_pickup_datetime': '2020-10-01 01:05:23', 'tpep_dropoff_datetime': '2020-10-01 02:05:23', 'passenger_count': 1, 'trip_distance': 0.1, 'PULocationID': 45, 'DOLocationID': 3, 'fare_amount': 10.0, 'tip_amount': 5.00}",
    "{'tpep_pickup_datetime': '2020-02-02 15:22:23', 'tpep_dropoff_datetime': '2020-02-03 15:44:23', 'passenger_count': 3, 'trip_distance': 3.25, 'PULocationID': 10, 'DOLocationID': 24, 'fare_amount': 5.05, 'tip_amount': 1.00}"
]
expected_types = {'DOLocationID': 'int', 'PULocationID': 'int', 'fare_amount': 'float', 'passenger_count': 'int', 'tip_amount': 'float', 'tpep_dropoff_datetime': 'timestamp', 'tpep_pickup_datetime': 'timestamp', 'trip_distance': 'float'}

test_df = spark.read.json(sc.parallelize(test_data))
test = transform_taxi_data(test_df)
test_types = {item[0]:item[1] for item in test.dtypes}

assert expected_types == test_types

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
shell_cmd("hdfs dfs -rm hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json/*")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

b'Deleted hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json/_SUCCESS'
b'Deleted hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json/part-00000-9f370f87-7731-4a6c-95ef-e39fda4a71cb-c000.json'
b'Deleted hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json/part-00000-be7dcba6-3674-4feb-804d-c9182e22e90c-c000.json'
b'Deleted hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json/part-00000-dab6c9ed-a352-48cb-b5e9-402b56f65a64-c000.json'
b'Deleted hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json/part-00000-f6e04c76-02bf-437a-9790-15cf365abefa-c000.json'
b'Deleted hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json/part-00001-9f370f87-7731-4a6c-95ef-e39fda4a71cb-c000.json'
b'Deleted hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json/part-00001-be7dcba6-3674-4feb-804d-c9182e22e90c-c000.json'
b'Deleted hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json/part-00001-dab6c9ed-a352-48cb-b5e9-402b56f65a64-c000.json'
b'Deleted hdfs:///tmp/data/nyc-taxi/taxi-

In [28]:
# Re run the ingestion using the functions with the transformation broken out
taxi_data_prefix = "s3://nyc-tlc/trip data/"
taxi_data_files = ["yellow_tripdata_2019-01.csv", "yellow_tripdata_2018-01.csv", "yellow_tripdata_2017-01.csv"]
for file_name in taxi_data_files: 
    taxi_data_path = f"{taxi_data_prefix}{file_name}"
    ingest_taxi_data_method(file_name)
    #ingest_taxi_data_transform(file_name)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
df = spark.read.json("hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json")
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

26137790

#### Lets try running the ingestion code on the other taxi data sets

In [30]:
# Try using the ingest code we created for yellow taxi for all the taxis
# This will fail because the datetime fields have different names across different servcies

taxi_data_prefix = "s3://nyc-tlc/trip data/"
taxi_data_files = ["green_tripdata_2020-01.csv", "fhv_tripdata_2020-01.csv", "fhvhv_tripdata_2020-01.csv"]
for file_name in taxi_data_files: 
    taxi_data_path = f"{taxi_data_prefix}{file_name}"
    ingest_taxi_data_transform(file_name)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

"cannot resolve '`tpep_pickup_datetime`' given input columns: [tolls_amount, congestion_surcharge, payment_type, extra, mta_tax, store_and_fwd_flag, lpep_dropoff_datetime, trip_type, PULocationID, improvement_surcharge, fare_amount, total_amount, tip_amount, ehail_fee, DOLocationID, lpep_pickup_datetime, RatecodeID, VendorID, trip_distance, passenger_count];;\n'Project ['tpep_pickup_datetime, 'tpep_dropoff_datetime, passenger_count#2458, trip_distance#2459, PULocationID#2456, DOLocationID#2457, fare_amount#2460, tip_amount#2463]\n+- Relation[VendorID#2451,lpep_pickup_datetime#2452,lpep_dropoff_datetime#2453,store_and_fwd_flag#2454,RatecodeID#2455,PULocationID#2456,DOLocationID#2457,passenger_count#2458,trip_distance#2459,fare_amount#2460,extra#2461,mta_tax#2462,tip_amount#2463,tolls_amount#2464,ehail_fee#2465,improvement_surcharge#2466,total_amount#2467,payment_type#2468,trip_type#2469,congestion_surcharge#2470] csv\n"
Traceback (most recent call last):
  File "<stdin>", line 17, in in

### Data modeling
Create a model for all taxi data, given that there are differences across the services in the kind of data collected

Commmon fields across all services:
* PULocationID
* DOLocationID

Fields we want to normalize across all services - this data is in all services but is named differently
* pickup datetime
* drop off datetime

Service specific fields. These are only in green or yellow data
* Passenger_count
* Trip_distance
* Fare_amount
* Tip_amount


### Lab 2.3 - Write different transformation functions for each taxi service type to match the following signature and schema:
Fields / Types:

* pickup_datetime Timestamp
* dropoff_datetime Timestamp
* passenger_count Integer
* fare_amount Float
* tip_amount Float
* PULocationID Integer
* DOLocationID Integer

`def transform_function(dataframe):  
    return transformed_dataframe
`

Once the transformation functions are done, rewrite `ingest_taxi_data` to use these new functions depending on the file being processed

In [17]:
def transform_yellow_taxi(df):
    return (df.withColumn("pickup_datetime", f.col("tpep_pickup_datetime").cast(t.TimestampType()))
        .withColumn("dropoff_datetime", f.col("tpep_dropoff_datetime").cast(t.TimestampType()))
        .withColumn("passenger_count", f.col("passenger_count").cast(t.IntegerType()))
        .withColumn("trip_distance", f.col("trip_distance").cast(t.FloatType()))
        .withColumn("fare_amount", f.col("fare_amount").cast(t.FloatType()))
        .withColumn("tip_amount", f.col("tip_amount").cast(t.FloatType()))
        .withColumn("PULocationID", f.col("PULocationID").cast(t.IntegerType()))
        .withColumn("DOLocationID", f.col("DOLocationID").cast(t.IntegerType())))
        
        
def transform_green_taxi(df):
    return (df.withColumn("pickup_datetime", f.col("lpep_pickup_datetime").cast(t.TimestampType()))
        .withColumn("dropoff_datetime", f.col("lpep_dropoff_datetime").cast(t.TimestampType()))
        .withColumn("passenger_count", f.col("passenger_count").cast(t.IntegerType()))
        .withColumn("trip_distance", f.col("trip_distance").cast(t.FloatType()))
        .withColumn("fare_amount", f.col("fare_amount").cast(t.FloatType()))
        .withColumn("tip_amount", f.col("tip_amount").cast(t.FloatType()))
        .withColumn("PULocationID", f.col("PULocationID").cast(t.IntegerType()))
        .withColumn("DOLocationID", f.col("DOLocationID").cast(t.IntegerType())))
 

def transform_fhv(df):
    return (df.withColumn("pickup_datetime", f.col("pickup_datetime").cast(t.TimestampType()))
        .withColumn("dropoff_datetime", f.col("dropoff_datetime").cast(t.TimestampType()))
        .withColumn("PULocationID", f.col("PULocationID").cast(t.IntegerType()))
        .withColumn("DOLocationID", f.col("DOLocationID").cast(t.IntegerType())))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Now that we have the ability to read all the taxi data into the same dataset, how will we be able to tell where the original data came from? The file name provides information including:
* Service type (yellow, green, etc)
* File date

We want to augment the taxi data with this information so we can refer back to it in analysis.

Is there other data we might want to augment the raw data with? Some things to consider:
* Additional fields that could help with analysis
* Metadata, like when the record was last updated

In [14]:
# Using matched groups, we can extract information from the taxi file names
TAXI_DATA_PATTERN = "(?P<service>[a-zA-Z0-9]+)_tripdata_(?P<year>[0-9]{4})-(?P<month>[0-9]{2}).csv"

def extract_file_info(file_name):
    m = re.match(TAXI_DATA_PATTERN, file_name)
    if m is not None:
        return (m.group(1), m.group(2), m.group(3))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
def ingest_taxi_data_multi_service(file_name, ingested_on):
    print(f"Processing {file_name}")
    (service, year, month) = extract_file_info(Path(file_name).name)
    input_df = spark.read.option('header', True).csv(file_name)
    
    if service == 'yellow':
        df_transform = transform_yellow_taxi(input_df)
    elif service == 'green':
        df_transform = transform_green_taxi(input_df)
    else:
        # FHV. What happens if there are more taxi services added?
        df_transform = transform_fhv(input_df)

    (df_transform
         .withColumn("service", f.lit(service))
         .withColumn("year", f.lit(year))
         .withColumn("month", f.lit(month))
         .withColumn("ingested_on", f.lit(ingest_timestamp))
         .write
         .mode("append")
         .json("hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json")
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
shell_cmd("hdfs dfs -rm -r hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

b'Deleted hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json'
b''

In [18]:
ingest_timestamp = datetime.strftime(datetime.now(), "%Y-%m-%d %H:%M:%S%z")
taxi_data_prefix = "s3://nyc-tlc/trip data/"
taxi_data_files = ["yellow_tripdata_2020-01.csv", "green_tripdata_2020-01.csv", "fhv_tripdata_2020-01.csv", "fhvhv_tripdata_2020-01.csv"]
for file_name in taxi_data_files: 
    taxi_data_path = f"{taxi_data_prefix}{file_name}"
    ingest_taxi_data_multi_service(taxi_data_path, ingest_timestamp)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Processing s3://nyc-tlc/trip data/yellow_tripdata_2020-01.csv

In [19]:
df_taxi_output = spark.read.json("hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df_taxi_output.groupby("service").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------+
|service|  count|
+-------+-------+
| yellow|6405008|
+-------+-------+

### Handling bad data
How to design for the inevitability of bad data  
Reference: https://blog.knoldus.com/apache-spark-handle-corrupt-bad-records/

In [38]:
bad_data = [
    "{'pickup_datetime': '2020-05-23 21:05:23', 'fare_amount': '0.05'}",
    "{'pickup_datetime': '2020-05-23 08:05:23', 'fare_amount': '10.05'}",
    "{'pickup_datetime': '2020-05-23 21:05:23', 'fare_amount}"
]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
corrupt_df = spark.read.json(sc.parallelize(bad_data), mode="PERMISSIVE", columnNameOfCorruptRecord="_corrupt_record")
corrupt_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+-------------------+
|     _corrupt_record|fare_amount|    pickup_datetime|
+--------------------+-----------+-------------------+
|                null|       0.05|2020-05-23 21:05:23|
|                null|      10.05|2020-05-23 08:05:23|
|{'pickup_datetime...|       null|               null|
+--------------------+-----------+-------------------+

In [40]:
corrupt_df = spark.read.json(sc.parallelize(bad_data), mode="DROPMALFORMED", columnNameOfCorruptRecord="_corrupt_record")
corrupt_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------------------+
|fare_amount|    pickup_datetime|
+-----------+-------------------+
|       0.05|2020-05-23 21:05:23|
|      10.05|2020-05-23 08:05:23|
+-----------+-------------------+

In [41]:
corrupt_df = spark.read.json(sc.parallelize(bad_data), mode="FAILFAST", columnNameOfCorruptRecord="_corrupt_record")
corrupt_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o900.json.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 15 in stage 69.0 failed 4 times, most recent failure: Lost task 15.3 in stage 69.0 (TID 903, ip-172-31-8-43.us-east-2.compute.internal, executor 6): org.apache.spark.SparkException: Malformed records are detected in schema inference. Parse Mode: FAILFAST.
	at org.apache.spark.sql.catalyst.json.JsonInferSchema$$anonfun$1$$anonfun$apply$1.apply(JsonInferSchema.scala:66)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema$$anonfun$1$$anonfun$apply$1.apply(JsonInferSchema.scala:53)
	at scala.collection.Iterator$$anon$12.nextCur(Iterator.scala:435)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:441)
	at scala.collection.Iterator$class.isEmpty(Iterator.scala:331)
	at scala.collection.AbstractIterator.isEmpty(Iterator.scala:1334)
	at scala.collection.TraversableOnce$class.reduceLeftOption(TraversableOnce.scala:203)
	at scala.collection.AbstractIterator.reduceL

### Lab 2.4 - Write an ingestion for the taxi zone lookup
File location - Yes, there is a space between taxi and the '_'  

s3://nyc-tlc/misc/taxi _zone_lookup.csv

`def ingest_taxi_lookup():`
1. Read taxi data
1. Cast to correct data types
1. Save to hdfs:///tmp/data/nyc-taxi/zone-lookup/output/section2/json

Refer back to Taxi Data page for more info: https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [21]:
def taxi_zone_transform(df):
    return (df.withColumn("LocationID", f.col("LocationID").cast(t.IntegerType()))
            .withColumn("Borough", f.col("Borough").cast(t.StringType()))
            .withColumn("Zone", f.col("Zone").cast(t.StringType()))
            .withColumn("service_zone", f.col("service_zone").cast(t.StringType())))

def ingest_taxi_lookup(ingest_timestamp):
    (spark.read
    .option("header", True)
    .csv("s3://nyc-tlc/misc/taxi _zone_lookup.csv")
    .transform(taxi_zone_transform)
    .withColumn("ingested_on", f.lit(ingest_timestamp))
    .write
    .mode("overwrite")
    .json("hdfs:///tmp/data/nyc-taxi/zone-lookup/output/section2/json"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
ingest_timestamp = datetime.strftime(datetime.now(), "%Y-%m-%d %H:%M:%S%z")
print(timer_method("ingest_taxi_lookup(ingest_timestamp)"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(0.9028045780000866, None)

# Break

# Weather data ingestion

NOAA GHCND dataset  
https://docs.opendata.aws/noaa-ghcn-pds/readme.html  

Scroll down to 'FORMAT OF “ghcnd-stations.txt” file' for the schema of the fixed-width stations data


In [22]:
ghcnd_stations_path = "https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt"
stations_s3 = f"s3://{MY_BUCKET_NAME}/data/ghcnd/stations/input/ghcnd_stations.txt"
stations_local = "hdfs:///tmp/data/ghcnd/stations/input/ghcnd-stations.txt"
stations_output = "hdfs:///tmp/data/ghcnd/stations/output/ghcnd-stations.txt"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# Workaround reading HTTPS -> HDFS, HTTPS -> S3 -> HDFS
# Spark cant read data directly from HTTP, so copy the file to S3 and read into a dataframe from there
# Then save the file to HDFS for further processing
ingest_timestamp = datetime.strftime(datetime.now(), "%Y-%m-%d %H:%M:%S%z")
resp = requests.get(ghcnd_stations_path)
if resp.status_code != 200:
    print("Couldn't get station data")
else:
    s3 = boto3.client('s3')
    res = s3.put_object(Body=resp.content, Bucket=MY_BUCKET_NAME, Key=f"data/ghcnd/stations/input/ghcnd_stations.txt")
    if res['ResponseMetadata']['HTTPStatusCode'] != 200:
        print(f"Unable to create ghcnd_stations.txt in s3, response {res['ResponseMetadata']['HTTPStatusCode']}")
    else:
        (spark
         .read
         .text(stations_s3)
         .write
         .format("text")
         .mode("overwrite")
         .save(stations_local))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
# Take a look at the stations file we just saved to HDFS
stations = spark.read.text(stations_local)
stations.show(10, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------------------------------------------------------+
|value                                                                                |
+-------------------------------------------------------------------------------------+
|ACW00011604  17.1167  -61.7833   10.1    ST JOHNS COOLIDGE FLD                       |
|ACW00011647  17.1333  -61.7833   19.2    ST JOHNS                                    |
|AE000041196  25.3330   55.5170   34.0    SHARJAH INTER. AIRP            GSN     41196|
|AEM00041194  25.2550   55.3640   10.4    DUBAI INTL                             41194|
|AEM00041217  24.4330   54.6510   26.8    ABU DHABI INTL                         41217|
|AEM00041218  24.2620   55.6090  264.9    AL AIN INTL                            41218|
|AF000040930  35.3170   69.0170 3366.0    NORTH-SALANG                   GSN     40930|
|AFM00040938  34.2100   62.2280  977.2    HERAT                                  40938|
|AFM00040948  34.5660   69.2120 

In [67]:
# Example of doing a substring transformation
(stations
    .withColumn("id", f.col("value").substr(0, 11))
    .drop("value")
).show(10, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+
|id         |
+-----------+
|ACW00011604|
|ACW00011647|
|AE000041196|
|AEM00041194|
|AEM00041217|
|AEM00041218|
|AF000040930|
|AFM00040938|
|AFM00040948|
|AFM00040990|
+-----------+
only showing top 10 rows

### Lab 2.5 - Build the ingestion for the weather stations data

Reference the fixed width schema provided under **FORMAT OF “ghcnd-stations.txt” file**   
https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt  
#### Important note - FORMAT OF is incorrect for: elevation - should be 7 char not 6

Create transformation code to convert the value column into the following schema:
* id String
* lat Float
* long Float
* elevation Float
* state String
* name String

Drop the value column, save the data in JSON format to s3://data-scale-oreilly/data/ghcnd/stations/output/section2_json 

In [29]:
def transform_stations_ps(stations_local):
    return (spark.read.text(stations_local)
     .withColumn("id", f.col("value").substr(0, 11))
     .withColumn("lat", f.col("value").substr(13, 8))
     .withColumn("long", f.col("value").substr(22, 9))
     .withColumn("elevation", f.col("value").substr(32, 7))
     .withColumn("state", f.col("value").substr(39, 2))
     .withColumn("name", f.col("value").substr(42, 30))
     .drop("value")
    )

def transform_stations_pd(stations_s3):
    return pd.read_fwf(stations_s3, 
                     [(0,10), (13, 20), (22, 30), (32, 38), (39, 40), (41, 71)],
                    names=["id", "lat", "long", "elevation", "state", "name"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
ps_run_time, stations_df = timer_method("transform_stations_ps(stations_local)")
pd_run_time, pd_stations_df = timer_method("transform_stations_pd(stations_s3)")
print(f"pyspark runtime: {ps_run_time} pandas runtime {pd_run_time}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### A key aspect of designing scalable systems is to be judicious about the data being stored and processed. 
#### The GHCND stations file contains data on stations across the US, but we are only interested in data near NYC

In [33]:
stations_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

115084

In [36]:
# Lets look at some performance tradeoffs between pyspark and pandas
# The .toPandas() function in pyspark can be convenient if you are familiar with pandas manipulations
# but this can quickly become very expensive as it collects all data on the driver to do the conversion.

def filter_ny_stations_pandas(stations_df):
    ny_stations = stations_df.filter("state == 'NY'")

    # filter down to just stations in NY in NYC. Lat of south Yonkers ~40.9124
    ny_pandas = ny_stations.toPandas()
    ny_pandas[ny_pandas.columns] = ny_pandas.apply(lambda x: x.str.strip())
    nyc_stations = ny_pandas[ny_pandas['lat'].apply(lambda x: float(x)) < 40.9124]
    res = spark.createDataFrame(nyc_stations)
    res.count()
    return res

def filter_ny_stations_pyspark(stations_df):
    print("Filtering stations to NY only")
    ny_stations = stations_df.filter("state == 'NY'")
    res = (ny_stations
            .withColumn("lat", f.col("lat").cast(t.FloatType()))
            .filter("lat < 40.9124"))
    res.count()
    return res

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
pd_run_time, pd_ny_stations = timer_method("filter_ny_stations_pandas(stations_df)")
print(f"runtime {pd_run_time}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

runtime 15.594902415999968

In [40]:
ps_run_time, ps_ny_stations = timer_method("filter_ny_stations_pyspark(stations_df)")
print(f"runtime {ps_run_time}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Filtering stations to NY only
runtime 0.3877436080001644

### Lab 2.6 - Write two ingest functions for the stations data, one using filter_ny_stations_pandas and the other using filter_ny_stations_pyspark. What do you notice about the differences?

The functions should:
* Read the station data from local or s3
* Transform the station data into columns from the fixed width format
* Use the above filter functions
* Write the output to stations_output as json in overwrite mode

In [41]:
def transform_station(stations):
    return (stations
     .withColumn("id", f.col("value").substr(0, 11))
     .withColumn("lat", f.col("value").substr(13, 8))
     .withColumn("long", f.col("value").substr(22, 9))
     .withColumn("elevation", f.col("value").substr(32, 7))
     .withColumn("state", f.col("value").substr(39, 2))
     .withColumn("name", f.col("value").substr(42, 30))
     .drop("value")
    )

def ingest_station_pandas():
    stations = spark.read.text(stations_local)
    (stations.transform(transform_station)
     .transform(filter_ny_stations_pandas)
     .coalesce(1)
     .write
     .mode("overwrite")
     .json(stations_output)
    )
    
def ingest_station_pyspark():
    stations = spark.read.text(stations_s3)
    (stations.transform(transform_station)
     .transform(filter_ny_stations_pyspark)
     .coalesce(1)
     .write
     .mode("overwrite")
     .json(stations_output)
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
#shell_cmd("hdfs dfs -rm -r hdfs:///tmp/data/test_pyspark")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
ps_result = timer_method("ingest_station_pyspark()")
pd_result = timer_method("ingest_station_pandas()")
print(f"pandas: {pd_result} pyspark: {ps_result}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Filtering stations to NY only
pandas: (0.9616486069999155, None) pyspark: (2.0233572219999587, None)

### Lab 2.7 - Case Study 1: Month over month, get the total count of of pickups per borough

In [6]:
taxiPath = "hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json/"
taxiLookupPath = "hdfs:///tmp/data/nyc-taxi/zone-lookup/output/section2/json/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
# Join boroughs
# Expected error cartesian join. most likely a carryover bug from 2.0
spark.conf.set("spark.sql.crossJoin.enabled", "false") #<-- default
df_taxi = spark.read.json(taxiPath)
df_taxi_lookup = spark.read.json(taxiLookupPath)

taxi_filtered = (df_taxi
 .filter(df_taxi.pickup_datetime.isNotNull())
 .filter(df_taxi.dropoff_datetime.isNotNull()))
taxi_pu = (taxi_filtered
.join(df_taxi_lookup
       .select("LocationID", "Borough")
       .withColumnRenamed("Borough", "PUBorough"), 
       df_taxi_lookup.LocationID == df_taxi.PULocationID))
taxi = (taxi_pu.join(df_taxi_lookup
       .select("LocationID", "Borough")
       .withColumnRenamed("Borough", "DOBorough"), 
       df_taxi_lookup.LocationID == taxi_pu.DOLocationID))
taxi_pu.show()
taxi.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'Detected implicit cartesian product for INNER join between logical plans\nJoin Inner, ((LocationID#3838L = PULocationID#3774L) && (LocationID#3838L = DOLocationID#3773L))\n:- Filter ((isnotnull(pickup_datetime#3791) && isnotnull(dropoff_datetime#3780)) && (((DOLocationID#3773L = PULocationID#3774L) && isnotnull(PULocationID#3774L)) && isnotnull(DOLocationID#3773L)))\n:  +- Relation[DOLocationID#3773L,PULocationID#3774L,RatecodeID#3775,SR_Flag#3776,VendorID#3777,congestion_surcharge#3778,dispatching_base_num#3779,dropoff_datetime#3780,extra#3781,fare_amount#3782,hvfhs_license_num#3783,improvement_surcharge#3784,lpep_dropoff_datetime#3785,lpep_pickup_datetime#3786,month#3787,mta_tax#3788,passenger_count#3789L,payment_type#3790,pickup_datetime#3791,service#3792,store_and_fwd_flag#3793,tip_amount#3794,tolls_amount#3795,total_amount#3796,... 5 more fields] json\n+- Project [LocationID#3838L, Borough#3837 AS PUBorough#3847]\n   +- Filter isnotnull(LocationID#3838L)\n      +- Relation[Boroug

In [24]:
spark.conf.set("spark.sql.crossJoin.enabled", "true")
df_taxi = spark.read.json(taxiPath)
df_taxi_lookup = spark.read.json(taxiLookupPath)

taxi_filtered = (df_taxi
 .filter(df_taxi.pickup_datetime.isNotNull())
 .filter(df_taxi.dropoff_datetime.isNotNull()))
taxi_pu = (taxi_filtered
.join(df_taxi_lookup
       .select("LocationID", "Borough")
       .withColumnRenamed("Borough", "PUBorough"), 
       df_taxi_lookup.LocationID == df_taxi.PULocationID))
taxi = (taxi_pu.join(df_taxi_lookup
       .select("LocationID", "Borough")
       .withColumnRenamed("Borough", "DOBorough"), 
       df_taxi_lookup.LocationID == taxi_pu.DOLocationID))
taxi_pu.explain()
taxi_pu.show()
taxi.explain()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan(isFinalPlan=false)
+- BroadcastHashJoin [PULocationID#806L], [LocationID#860L], Inner, BuildRight
   :- Project [DOLocationID#805L, PULocationID#806L, RatecodeID#807, VendorID#808, congestion_surcharge#809, dropoff_datetime#810, extra#811, fare_amount#812, improvement_surcharge#813, ingested_on#814, month#815, mta_tax#816, passenger_count#817L, payment_type#818, pickup_datetime#819, service#820, store_and_fwd_flag#821, tip_amount#822, tolls_amount#823, total_amount#824, tpep_dropoff_datetime#825, tpep_pickup_datetime#826, trip_distance#827, year#828]
   :  +- Filter ((isnotnull(pickup_datetime#819) && isnotnull(dropoff_datetime#810)) && isnotnull(PULocationID#806L))
   :     +- FileScan json [DOLocationID#805L,PULocationID#806L,RatecodeID#807,VendorID#808,congestion_surcharge#809,dropoff_datetime#810,extra#811,fare_amount#812,improvement_surcharge#813,ingested_on#814,month#815,mta_tax#816,passenger_count#817L,payment_type#818,pickup_datetime#819,se

In [9]:
def get_monthly_totals_pyspark(taxiPath, taxiLookupPath):
    taxi = spark.read.json(taxiPath)
    taxi_lookup = spark.read.json(taxiLookupPath)
    taxi_filtered = (taxi
     .filter(taxi.pickup_datetime.isNotNull())
     .filter(taxi.dropoff_datetime.isNotNull()))
                     
    groupDF = taxi_filtered.join(taxi_lookup, taxi_filtered.PULocationID == taxi_lookup.LocationID)
    groupDF.select("ingested_on").show() # expected error
    return groupDF

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
print(timer_method("get_monthly_totals_pyspark(taxiPath, taxiLookupPath)"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

"Reference 'ingested_on' is ambiguous, could be: ingested_on, ingested_on.;"
Traceback (most recent call last):
  File "<stdin>", line 57, in timer_method
  File "/usr/lib64/python3.7/timeit.py", line 232, in timeit
    return Timer(stmt, setup, timer, globals).timeit(number)
  File "/usr/lib64/python3.7/timeit.py", line 176, in timeit
    timing = self.inner(it, self.timer)
  File "<timeit-src>", line 6, in inner
  File "<stdin>", line 9, in get_monthly_totals_pyspark
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 1326, in select
    jdf = self._jdf.select(self._jcols(*cols))
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: "Reference 'ingested_on' is ambiguous, co

In [10]:
def get_monthly_totals_pandas(taxiPath, taxiLookupPath):
    taxi = pd.read_json(taxiPath)
    taxi_lookup = pd.read_json(taxiLookupPath)
    taxi_filtered = tax.dropna(subset=['pickup_datetime', 'dropoff_datetime'])
    
    groupDF = taxi_filtered.join(taxi_lookup.set_index('LocationID'), on='PULocationID')
    groupDF['pickup_month'] = pd.to_datetime(groupDF['pickup_datetime'], format='%m%Y')
    groupDF = groupDF.groupby('pickup_month', 'borough').agg('count').sort_values(by=['count', 'borough'], ascending=[False, True])
    groupDF
    return groupDF

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
print(timer_method("get_monthly_totals_pandas(taxiPath, taxiLookupPath)"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

pyarrow and local java libraries required for HDFS
Traceback (most recent call last):
  File "<stdin>", line 57, in timer_method
  File "/usr/lib64/python3.7/timeit.py", line 232, in timeit
    return Timer(stmt, setup, timer, globals).timeit(number)
  File "/usr/lib64/python3.7/timeit.py", line 176, in timeit
    timing = self.inner(it, self.timer)
  File "<timeit-src>", line 6, in inner
  File "<stdin>", line 2, in get_monthly_totals_pandas
  File "/tmp/1603424693133-0/lib/python3.7/site-packages/pandas/util/_decorators.py", line 199, in wrapper
    return func(*args, **kwargs)
  File "/tmp/1603424693133-0/lib/python3.7/site-packages/pandas/util/_decorators.py", line 296, in wrapper
    return func(*args, **kwargs)
  File "/tmp/1603424693133-0/lib/python3.7/site-packages/pandas/io/json/_json.py", line 594, in read_json
    path_or_buf, encoding=encoding, compression=compression
  File "/tmp/1603424693133-0/lib/python3.7/site-packages/pandas/io/common.py", line 222, in get_filepath_or

In [12]:
def get_monthly_totals_pandas(taxiPath, taxiLookupPath):
    taxiPyspark = spark.read.json(taxiPath)
    taxiLookupPySpark = spark.read.json(taxiLookupPath)
    
    taxi = taxiPyspark.toPandas()
    taxi_lookup = taxiLookupPyspark.toPandas()
    taxi_filtered = taxi.dropna(subset=['pickup_datetime', 'dropoff_datetime'])
    
    groupDF = taxi_filtered.join(taxi_lookup.set_index('LocationID'), on='PULocationID')
    groupDF['pickup_month'] = pd.to_datetime(groupDF['pickup_datetime'], format='%m%Y')
    groupDF = groupDF.groupby('pickup_month', 'borough').agg('count').sort_values(by=['count', 'borough'], ascending=[False, True])
    groupDF
    return groupDF

def get_monthly_totals_pyspark(taxiPath, taxiLookupPath):
    taxi = spark.read.json(taxiPath)
    taxi_lookup = spark.read.json(taxiLookupPath)
    
    taxi_filtered = (taxi
     .filter(taxi.pickup_datetime.isNotNull())
     .filter(taxi.dropoff_datetime.isNotNull()))
                     
    groupDF = taxi_filtered.join(taxi_lookup.select("Borough", "LocationID"), taxi_filtered.PULocationID == taxi_lookup.LocationID)
    groupDF = groupDF.withColumn("pickup_month", f.date_format("pickup_datetime", "yyyyMM"))
    groupDF = groupDF.groupBy("pickup_month", "borough").count().orderBy(f.desc("pickup_month"), f.desc("count"), "borough")
    groupDF.show()
    return groupDF

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
print(timer_method("get_monthly_totals_pandas(taxiPath, taxiLookupPath)"))

In [84]:
print(timer_method("get_monthly_totals_pyspark(taxiPath, taxiLookupPath)"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------------+--------+
|pickup_month|      borough|   count|
+------------+-------------+--------+
|      202101|    Manhattan|       3|
|      202007|    Manhattan|       6|
|      202006|    Manhattan|       1|
|      202005|    Manhattan|       5|
|      202004|    Manhattan|       1|
|      202003|    Manhattan|       5|
|      202002|    Manhattan|      34|
|      202002|       Queens|       9|
|      202002|     Brooklyn|       3|
|      202002|        Bronx|       1|
|      202002|      Unknown|       1|
|      202001|    Manhattan|14817800|
|      202001|     Brooklyn| 5628274|
|      202001|       Queens| 4509457|
|      202001|        Bronx| 2536611|
|      202001|      Unknown| 1615247|
|      202001|Staten Island|  260258|
|      202001|          EWR|    3779|
|      201912|    Manhattan|     129|
|      201912|       Queens|      17|
+------------+-------------+--------+
only showing top 20 rows

(46.01737120200005, DataFrame[pickup_month: string, borough: s

In [85]:
## Expected error for maxResultSize: This won't work. Could try the subsequent cells
## Those restart the state of the notebook and don't work as expected
## Need to restart the cluster and edit the Software config with: [{"classification":"spark-defaults", "properties":{"spark.driver.maxResultSize":"5G", "spark.ui.killEnabled":"true"}, "configurations":[]}]
## Then need to reun the taxi and taxi lookup ingests
## Run -> Run All Above Selected Cell
## Second expected error for {"msg":"requirement failed: Session isn't active."} and will hang. Driver node ran out of mem. Will need to go and upscale
spark.conf.set("spark.driver.maxResultSize", "5G")
print(spark.conf.get('spark.driver.maxResultSize'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5G

In [67]:
%%configure -f
{"conf":{"spark.driver.maxResultSize":"5G"}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1603063175185_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1603063175185_0002,pyspark,idle,Link,Link,✔


In [94]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1603063175185_0003,pyspark,dead,Link,Link,✔


In [8]:
def get_monthly_totals_concat_pandas(taxiPath, taxiLookupPath):
    taxiPyspark = spark.read.json(taxiPath)
    taxiLookupPySpark = spark.read.json(taxiLookupPath)
    
    taxi = taxiPyspark.toPandas()
    taxi_lookup = taxiLookupPyspark.toPandas()
    taxi_filtered = taxi.dropna(subset=['pickup_datetime', 'dropoff_datetime'])
    
    groupDF = taxi_filtered.join(taxi_lookup.set_index('LocationID'), on='PULocationID')
    groupDF['pickup_month'] = groupDF['month'] + groupDF['year']
    groupDF = groupDF.groupby('pickup_month', 'borough').agg('count').sort_values(by=['count', 'borough'], ascending=[False, True])
    groupDF
    return groupDF
    
def get_monthly_totals_concat_pyspark(taxiPath, taxiLookupPath):
    taxi = spark.read.json(taxiPath)
    taxi_lookup = spark.read.json(taxiLookupPath)
    taxi_filtered = (taxi
     .filter(taxi.pickup_datetime.isNotNull())
     .filter(taxi.dropoff_datetime.isNotNull()))
         
    groupDF = taxi_filtered.withColumn("pickup_month", f.concat("month", "year")).select("pickup_datetime", "borough", "pickup_month")
    groupDF = groupDF.groupBy("pickup_month", "borough").count().orderBy(f.desc("pickup_month"), f.desc("count"), "borough")
    groupDF.show()
    return groupDF

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
print(timer_method("get_monthly_totals_concat_pandas(taxiPath, taxiLookupPath)"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Interpreter died:


ERROR:fake_shell:execute_reply
Traceback (most recent call last):
  File "/tmp/8024848134341106643", line 229, in execute
    exec(code, global_dict)
  File "<stdin>", line 1, in <module>
  File "<stdin>", line 57, in timer_method
  File "/usr/lib64/python3.7/timeit.py", line 232, in timeit
    return Timer(stmt, setup, timer, globals).timeit(number)
  File "/usr/lib64/python3.7/timeit.py", line 176, in timeit
    timing = self.inner(it, self.timer)
  File "<timeit-src>", line 6, in inner
NameError: name 'taxiPath' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/8024848134341106643", line 318, in execute_request
    result = node.execute()
  File "/tmp/8024848134341106643", line 233, in execute
    raise ExecutionError(sys.exc_info())
ExecutionError: (<class 'NameError'>, NameError("name 'taxiPath' is not defined"), <traceback object at 0x7fd1b192bf00>)



In [9]:
print(timer_method("get_monthly_totals_concat_pyspark(taxiPath, taxiLookupPath)"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

"cannot resolve '`borough`' given input columns: [tpep_dropoff_datetime, extra, service, month, improvement_surcharge, tip_amount, VendorID, passenger_count, pickup_datetime, store_and_fwd_flag, DOLocationID, trip_distance, year, ingested_on, dropoff_datetime, total_amount, tolls_amount, RatecodeID, PULocationID, pickup_month, tpep_pickup_datetime, mta_tax, congestion_surcharge, fare_amount, payment_type];;\n'Project [pickup_datetime#143, 'borough, pickup_month#193]\n+- Project [DOLocationID#129L, PULocationID#130L, RatecodeID#131, VendorID#132, congestion_surcharge#133, dropoff_datetime#134, extra#135, fare_amount#136, improvement_surcharge#137, ingested_on#138, month#139, mta_tax#140, passenger_count#141L, payment_type#142, pickup_datetime#143, service#144, store_and_fwd_flag#145, tip_amount#146, tolls_amount#147, total_amount#148, tpep_dropoff_datetime#149, tpep_pickup_datetime#150, trip_distance#151, year#152, concat(month#139, year#152) AS pickup_month#193]\n   +- Filter isnotnull

### Lab 2.8 - Case Study 2: Month over month, get the borough with the most amount of pickups per month

Add pandas

In [13]:
def get_most_pickups_per_month_pandas(taxiPath, taxiLookupPath):
    inputDF = get_monthly_totals_pandas(taxiPath, taxiLookupPath)
    firstDF = inputDF.groupby('pickup_month').sort_values(by=['pickup_month', 'count'], ascending=[True, False]).first()
    firstDF
    return firstDF

def get_most_pickups_per_month_pyspark(taxiPath, taxiLookupPath):
    inputDF = get_monthly_totals_pyspark(taxiPath, taxiLookupPath)
    firstDF = inputDF.groupBy("pickup_month").orderBy(f.desc("pickup_month"), f.desc("count")).agg(f.first("borough"))
    firstDF.explain()
    firstDF.show()
    return firstDF

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
print(timer_method("get_most_pickups_per_month_pandas(taxiPath, taxiLookupPath)"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Interpreter died:


ERROR:fake_shell:execute_reply
Traceback (most recent call last):
  File "/tmp/5988482819598318708", line 229, in execute
    exec(code, global_dict)
  File "<stdin>", line 1, in <module>
  File "<stdin>", line 57, in timer_method
  File "/usr/lib64/python3.7/timeit.py", line 232, in timeit
    return Timer(stmt, setup, timer, globals).timeit(number)
  File "/usr/lib64/python3.7/timeit.py", line 176, in timeit
    timing = self.inner(it, self.timer)
  File "<timeit-src>", line 6, in inner
NameError: name 'taxiPath' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/5988482819598318708", line 318, in execute_request
    result = node.execute()
  File "/tmp/5988482819598318708", line 233, in execute
    raise ExecutionError(sys.exc_info())
ExecutionError: (<class 'NameError'>, NameError("name 'taxiPath' is not defined"), <traceback object at 0x7f6f559d6e10>)
ERROR:fake_shell:execute_repl

In [91]:
print(timer_method("get_most_pickups_per_month_pyspark(taxiPath, taxiLookupPath)"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from https://172.31.6.49:18888/sessions/2/statements/90 with error payload: {"msg":"requirement failed: Session isn't active."}


In [92]:
def get_most_pickups_per_month_window_pyspark(taxiPath, taxiLookupPath):
    from pyspark.sql import Window
    inputDF = get_monthly_totals_pyspark(taxiPath, taxiFilteredPath)
    win = Window.partitionBy("borough", "pickup_month").orderBy(f.desc("count"))
    firstDF = inputDF.withColumn("row_num", f.row_number().over(win)).where("row_num == 1")
    firstDF = firstDF.orderBy(f.desc("count"), f.desc("pickup_month"))
    firstDF.explain()
    firstDF.show()
    return firstDF

VBox()

An error was encountered:
Session 2 unexpectedly reached final status 'dead'. See logs:
stdout: 
#
# There is insufficient memory for the Java Runtime Environment to continue.
# Native memory allocation (mmap) failed to map 2498269184 bytes for committing reserved memory.
# An error report file with more information is saved as:
# /tmp/hs_err_pid20386.log

stderr: 
20/10/19 00:18:13 INFO TaskSetManager: Finished task 31.0 in stage 118.0 (TID 2090) in 3819 ms on ip-172-31-8-43.us-east-2.compute.internal (executor 28) (31/76)
20/10/19 00:18:13 INFO BlockManagerInfo: Removed taskresult_2090 on ip-172-31-8-43.us-east-2.compute.internal:40247 in memory (size: 20.3 MB, free: 4.8 GB)
20/10/19 00:18:13 INFO BlockManagerInfo: Added taskresult_2091 in memory on ip-172-31-4-197.us-east-2.compute.internal:37927 (size: 20.4 MB, free: 4.7 GB)
20/10/19 00:18:13 INFO TaskSetManager: Starting task 47.0 in stage 118.0 (TID 2107, ip-172-31-4-197.us-east-2.compute.internal, executor 27, partition 47, NODE

In [15]:
print(timer_method("get_most_pickups_per_month_window_pyspark(taxiPath, taxiLookupPath)"))

VBox()

An error was encountered:
Session 4 unexpectedly reached final status 'error'. See logs:
stdout: 

stderr: 
20/10/23 04:20:14 INFO TaskSetManager: Starting task 7.0 in stage 10.0 (TID 113, ip-172-31-11-79.ec2.internal, executor 13, partition 7, NODE_LOCAL, 8375 bytes)
20/10/23 04:20:14 INFO BlockManagerInfo: Added broadcast_17_piece0 in memory on ip-172-31-8-57.ec2.internal:35637 (size: 6.1 KB, free: 5.3 GB)
20/10/23 04:20:14 INFO BlockManagerInfo: Added broadcast_17_piece0 in memory on ip-172-31-11-79.ec2.internal:36177 (size: 6.1 KB, free: 5.3 GB)
20/10/23 04:20:15 INFO BlockManagerInfo: Added broadcast_17_piece0 in memory on ip-172-31-11-79.ec2.internal:41257 (size: 6.1 KB, free: 5.3 GB)
20/10/23 04:20:15 INFO BlockManagerInfo: Added broadcast_16_piece0 in memory on ip-172-31-8-57.ec2.internal:35637 (size: 28.4 KB, free: 5.3 GB)
20/10/23 04:20:15 INFO BlockManagerInfo: Added broadcast_16_piece0 in memory on ip-172-31-11-79.ec2.internal:36177 (size: 28.4 KB, free: 5.3 GB)
20/10/23 04

### Lab - 2.9 Run and time the overall pipeline

In [75]:
# Reset notebook kernel
def ingest_main():
    ingest_taxi_data_multi_service("s3://nyc-tlc/trip data/yellow_tripdata_2020-01.csv")
    ingest_taxi_lookup("s3://nyc-tlc/misc/taxi _zone_lookup.csv")
    get_most_pickups_per_month_window_pyspark()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
print(timer_method("ingest_main"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'timer_method' is not defined
Traceback (most recent call last):
NameError: name 'timer_method' is not defined

